# Partie 1: Classification de Documents

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

2023-04-24 19:08:26.593658: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 19:08:27.016426: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-24 19:08:27.025255: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-24 19:08:27.025281: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

## CNN

### Chargement des Données

In [2]:
# Données d'entrainement
train_data_complete = pd.read_csv("../data/allocine_genres_train.csv", sep=",")
train_data = train_data_complete[["titre", "synopsis", "genre"]]

# Données de test/validation
test_data_complete = pd.read_csv("../data/allocine_genres_test.csv", sep=",")
test_data = test_data_complete[["titre", "synopsis", "genre"]]

Lister les classes et leur associer un identifiant unique.

In [3]:
# Liste des genres
genre_name = sorted(train_data.genre.unique().flatten())
print("Genres:", genre_name)
print("Nombre d'exemplaires:", len(train_data))

# Identifiant unique par genre
genre_index = {genre_name[i]:i for i in range(len(genre_name))}
genre_index

Genres: ['biopic', 'comédie', 'documentaire', 'drame', 'historique', 'horreur', 'policier', 'romance', 'science fiction']
Nombre d'exemplaires: 2875


{'biopic': 0,
 'comédie': 1,
 'documentaire': 2,
 'drame': 3,
 'historique': 4,
 'horreur': 5,
 'policier': 6,
 'romance': 7,
 'science fiction': 8}

Remplacer les genres par la valeur numérique associée.

In [4]:
train_data = train_data.replace({"genre": genre_index})
train_data.head()

,titre,synopsis,genre
0,Le Crime de l' Orient - Express,"En visite à Istanbul , le célèbre détective be...",6
1,12 hommes en colère,Un jeune homme d' origine modeste est accusé d...,3
2,Après moi le bonheur,"Lorsque Marie-Laure , mère de quatre jeunes en...",3
3,Les Lumières de la ville,Un vagabond s’ éprend d’ une belle et jeune ve...,7
4,Les Chemins de la dignité,"L' histoire vraie de Carl Brashear , premier A...",0


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data[["titre", "synopsis"]],
                                                    train_data[["genre"]],
                                                    test_size=0.2,
                                                    random_state=12, # Random seed for shuffle
                                                    shuffle=True)

On combine le titre et le synopsispour pouvoir les vectoriser par la suite.

In [6]:
X_train_titre = X_train.titre
X_train = X_train_titre + " " + X_train.synopsis

In [7]:
X_train.head()

2447    Le Coeur des hommes Alex , Antoine , Jeff et M...
2741    Sliver Dirigeante d' une maison d' édition , C...
2051    Grande-Synthe Crise migratoire , pollution ind...
474     Dallas Buyers Club 1986 , Dallas , Texas , une...
1037    Resident Evil : Chapitre Final Alice , seule s...
dtype: object

In [8]:
y_train.head()

,genre
2447,7
2741,6
2051,2
474,0
1037,5


In [9]:
print(X_train.shape)
print(y_train.shape)

(2300,)
(2300, 1)


## Indexation du Vocabulaire

In [10]:
def get_vectorizer(documents, max_voc_size=8000, max_seq_length= 200, batch_size=64):
	vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_voc_size, output_sequence_length=max_seq_length)
	# Création du jeu de données à partir de X_train et constitution de lots de 128 instances
	text_ds = tf.data.Dataset.from_tensor_slices(documents).batch(batch_size)
	# Création du vocabulaire à partir des données d'entrée
	vectorizer.adapt(text_ds)
	return vectorizer

In [11]:
keras_vectorizer = get_vectorizer(X_train)

2023-04-24 19:08:34.534061: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-24 19:08:34.534130: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-24 19:08:34.534169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fabien): /proc/driver/nvidia/version does not exist
2023-04-24 19:08:34.534649: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
voc = keras_vectorizer.get_vocabulary()
print(len(voc))

8000


In [13]:
voc[:10]

['', '[UNK]', 'de', 'la', 'et', 'le', 'à', 'un', 'une', 'les']

In [14]:
word_index = dict(zip(voc, range(len(voc))))

In [15]:
print("Texte initial:", X_train.iloc[1])
output = keras_vectorizer([X_train.iloc[1]])
print("Vocabulaire dans le texte (15 premiers items):")
for v in output.numpy()[0, :15]:
    print(v, keras_vectorizer.get_vocabulary()[v])

Texte initial: Sliver Dirigeante d' une maison d' édition , Carly Norris s' installe dans un building ultra moderne de New York : le « Sliver » . Son emménagement se déroule sans encombre si ce n' est le désagréable accueil que lui réserve un voisin . Un frisson parcourt la nuque de Carly lorsqu' il lui annonce qu' elle ressemble étonnamment à la précédente locataire , qui s' est suicidée . Intriguée , Carly espère en savoir un peu plus mais Gus décède lui aussi . C' est alors qu' elle fait la connaissance de ses autres voisins : Zeke et Jack . Elle constate avec amusement que ces deux hommes ne s' apprécient guère . Un malaise s' installe néanmoins . Carly se sent constamment épiée . Sharon Stone , au sommet de sa réputation , illumine ce thriller primé aux MTV Awards .
Vocabulaire dans le texte (15 premiers items):
7771 sliver
1 [UNK]
16 d
8 une
140 maison
16 d
3538 édition
3440 carly
1 [UNK]
37 s
608 installe
17 dans
7 un
1 [UNK]
2659 ultra


### Chargement de Plongements de Mots Pré-entraînés

In [20]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("../embedding/frWiki_no_phrase_no_postag_700_cbow_cut100.bin", binary=True, unicode_errors="ignore")
model.most_similar("bonjour")

[('merci', 0.7507892847061157),
 ('bonsoir', 0.7450243830680847),
 ('votre', 0.5642200112342834),
 ('vous', 0.5538792014122009),
 ('remercier', 0.5396129488945007),
 ('avance', 0.5288880467414856),
 ('discuter', 0.5033395886421204),
 ('je', 0.49339333176612854),
 ('désoler', 0.4899965822696686),
 ('ici', 0.4887441396713257)]

In [23]:
def load_embeddings(embeddings_file):
    embeddings_index = {}
    with open(embeddings_file, "r", encoding="utf8") as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embeddings_index[word] = coefs
    print(f'{len(embeddings_index)} vecteurs de mots ont été lus')
    return embeddings_index

In [24]:
m6_embeddings = load_embeddings("../embedding/frWiki_no_phrase_no_postag_700_cbow_cut100.bin")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xab in position 15: invalid start byte